# Census- Employment Status Data Template

In [1]:
import pandas as pd
import requests

In [2]:
#Census Subject Table API for Employment Status data within Unified School Districts in California for 2019
url="https://api.census.gov/data/2019/acs/acs1/subject?get=group(S2301)&for=school%20district%20(unified)&in=state:06"

In [3]:
#Request for HTTP Data from Census API, which is working <Response [200]>
response = requests.get(url)

In [4]:
#Resetting data from API Data for future formatting, lists data in one column
response_json = response.json()

In [5]:
#Places data in a dataframe and drops index column 0 (with headers), which has 120 schools and 564 columns of variables
unifiedschool_df = pd.DataFrame(response_json,columns=response_json[0]).drop(0)

In [6]:
#View School District to view column titles
unifiedschool_df.loc[unifiedschool_df["NAME"]=="Los Angeles Unified School District, California"]

,GEO_ID,NAME,S2301_C01_001E,S2301_C01_001EA,S2301_C01_001M,S2301_C01_001MA,S2301_C01_002E,S2301_C01_002EA,S2301_C01_002M,S2301_C01_002MA,...,S2301_C04_034E,S2301_C04_034EA,S2301_C04_034M,S2301_C04_034MA,S2301_C04_035E,S2301_C04_035EA,S2301_C04_035M,S2301_C04_035MA,state,school district (unified)
26,9700000US0622710,"Los Angeles Unified School District, California",3871485,None,18648,None,230952,None,6875,None,...,5.1,None,0.5,None,3.7,None,0.4,None,06,22710


## _Format Data_

In [7]:
#Census Subject Table Variables for Employment Status data
variableurl = "https://api.census.gov/data/2019/acs/acs1/subject/variables.json"

In [8]:
#Request for HTTP Data from Census API and reset data
variables_json = requests.get(variableurl).json()

In [9]:
#View variable for column title
variables_json["variables"]["S2301_C01_001E"]

{'label': 'Estimate!!Total!!Population 16 years and over',
 'concept': 'EMPLOYMENT STATUS',
 'predicateType': 'int',
 'group': 'S2301',
 'limit': 0,
 'attributes': 'S2301_C01_001EA,S2301_C01_001M,S2301_C01_001MA'}

In [10]:
#Find and replace all columns with variable titles
new_labels = []
for col in unifiedschool_df.columns:
    label = variables_json["variables"].get(col)
    if label is not None:
        label = label['label']
    else:
        if col[-2:] == 'EA':
            label = variables_json["variables"].get(col[:-1])
            label = label['label'] + "||Annotation"
        elif col[-1] == 'M':
            label = variables_json["variables"].get(col[:-1]+'E')
            label = label['label'] + "||MarginOfError"
        elif col[-2:] == 'MA':
            label = variables_json["variables"].get(col[:-2]+'E')
            label = label['label'] + "||MarginOfErrorAnnotation"
    new_labels.append(label)  

In [11]:
#Find any columns without titles
new_labels  

['Geography',
 None,
 'Estimate!!Total!!Population 16 years and over',
 'Estimate!!Total!!Population 16 years and over||Annotation',
 'Estimate!!Total!!Population 16 years and over||MarginOfError',
 'Estimate!!Total!!Population 16 years and over||MarginOfErrorAnnotation',
 'Estimate!!Total!!Population 16 years and over!!AGE!!16 to 19 years',
 'Estimate!!Total!!Population 16 years and over!!AGE!!16 to 19 years||Annotation',
 'Estimate!!Total!!Population 16 years and over!!AGE!!16 to 19 years||MarginOfError',
 'Estimate!!Total!!Population 16 years and over!!AGE!!16 to 19 years||MarginOfErrorAnnotation',
 'Estimate!!Total!!Population 16 years and over!!AGE!!20 to 24 years',
 'Estimate!!Total!!Population 16 years and over!!AGE!!20 to 24 years||Annotation',
 'Estimate!!Total!!Population 16 years and over!!AGE!!20 to 24 years||MarginOfError',
 'Estimate!!Total!!Population 16 years and over!!AGE!!20 to 24 years||MarginOfErrorAnnotation',
 'Estimate!!Total!!Population 16 years and over!!AGE!!2

In [12]:
#Change column titles for columns labeled "None" 
assert len(new_labels) == len(unifiedschool_df.columns)

In [13]:
#Confirm the number of columns without titles
sum([1 for x in new_labels if x is None])

3

In [14]:
#Setup new Labels of columns labeled "None"
new_labels[1] = 'NAME'
new_labels[-2] = 'STATE'
new_labels[-1] = 'SCHOOL DISTRICT (UNIFIED)'

In [15]:
#Create new labels of columns labeled "None"
unifiedschool_df.columns = new_labels

In [16]:
#Find all columns without "Annotation" in column title
#[col for col in unifiedschool_df.columns if "Annotation" not in col]

In [17]:
#Create a new dataframe for data without the columns that have "Annotation" in the title, which is 284 columns
without_annotation = unifiedschool_df[[col for col in unifiedschool_df.columns if "Annotation" not in col]].copy()

In [18]:
#Find all columns without "MarginOfError" in column title
#[col for col in without_annotation_df.columns if "MarginOfError" not in col]

In [19]:
#Create a new dataframe for data without the columns that have "MarginOfError" in the title, which is 144 columns
withoutmarginerror = without_annotation[[col for col in without_annotation.columns if 'MarginOfError' not in col]].copy()

In [20]:
#Find all columns without "Labor Force Participation Rate" in column title
#[col for col in withoutmarginerror.columns if "Labor Force Participation Rate" not in col]

In [21]:
#Create a new dataframe for data without the columns that have "Labor Force Participation Rate" in the title, which is 109 columns
withoutlaborforce = withoutmarginerror[[col for col in withoutmarginerror.columns if 'Labor Force Participation Rate' not in col]].copy()

In [22]:
#Find all columns without "Sex" in column title
#[col for col in withoutlaborforce.columns if "SEX" not in col]

In [23]:
#Create a new dataframe for data without the columns that have "Sex" in the title, which is 91 columns
withoutsex = withoutlaborforce[[col for col in withoutlaborforce.columns if 'SEX' not in col]].copy()

In [24]:
#Find all columns without "Poverty Status" in column title
#[col for col in withoutsex.columns if "POVERTY STATUS" not in col]

In [25]:
#Create a new dataframe for data without the columns that have "Poverty Status" in the title, which is 85 columns
withoutps = withoutsex[[col for col in withoutsex.columns if 'POVERTY STATUS' not in col]].copy()

In [26]:
#Find all columns without "Disability Status" in column title
#[col for col in withoutps.columns if "DISABILITY STATUS" not in col]

In [27]:
#Create a new dataframe for data without the columns that have "Disability Status" in the title, which is 82 columns
withoutds = withoutps[[col for col in withoutps.columns if 'DISABILITY STATUS' not in col]].copy()

In [28]:
#Find all columns without "Educational Attainment" in column title
#[col for col in withoutds.columns if "EDUCATIONAL ATTAINMENT" not in col]

In [29]:
#Create a new dataframe for data without the columns that have "Educational Attainment" in the title, which is 67 columns
withoutea = withoutds[[col for col in withoutds.columns if 'EDUCATIONAL ATTAINMENT' not in col]].copy()

In [30]:
#Find all columns without "Age" in column title
#[col for col in withoutea.columns if "AGE" not in col]

In [31]:
#Create a new dataframe for data without the columns that have "Age" in the title, which is 37 columns
withoutage = withoutea[[col for col in withoutea.columns if 'AGE' not in col]].copy()

In [32]:
#Format to rename School District Name
def cleandistrict(NAME):
    return NAME.replace(", California","")

In [33]:
#Apply formatting condition to School District Name
withoutage['NAME'].apply(cleandistrict)

1               Redlands Unified School District
2      West Contra Costa Unified School District
3              Riverside Unified School District
4                Rowland Unified School District
5        Sacramento City Unified School District
                         ...                    
116               Novato Unified School District
117              Oakland Unified School District
118            Oceanside Unified School District
119          Twin Rivers Unified School District
120              Turlock Unified School District
Name: NAME, Length: 120, dtype: object

In [34]:
#Create new School District name column with formatting titled District
withoutage['District']= withoutage['NAME'].apply(cleandistrict)

In [35]:
#Find length of Geographt column contents, which is 16
withoutage['Geography'].apply(len).unique()

#Pull a geography
geo = "9700000US0600158"

#Find NCESDist ID
geo.split("US")[1]

'0600158'

In [36]:
#Format to seperate Census Geography code (the state and district ID combined), 
def splitGeo(geo):
    return geo.split("US")[1]

In [37]:
#Apply formatting condition to Geography
withoutage['Geography'].apply(splitGeo)

1      0632070
2      0632550
3      0633150
4      0633750
5      0633840
        ...   
116    0627720
117    0628050
118    0628250
119    0601332
120    0600158
Name: Geography, Length: 120, dtype: object

In [38]:
##Create new Geography name column with formatting titled NCESDist
withoutage['NCESDist']= withoutage['Geography'].apply(splitGeo)

In [39]:
#Add Year column
withoutage['year']= "2019"

In [40]:
withoutage.head()

,Geography,NAME,Estimate!!Total!!Population 16 years and over,Estimate!!Total!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Estimate!!Total!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Estimate!!Total!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Estimate!!Total!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Estimate!!Total!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Estimate!!Total!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,...,Estimate!!Unemployment rate!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Estimate!!Unemployment rate!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Estimate!!Unemployment rate!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!Hispanic or Latino origin (of any race),"Estimate!!Unemployment rate!!Population 16 years and over!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino",Estimate!!Unemployment rate!!Population 20 to 64 years,STATE,SCHOOL DISTRICT (UNIFIED),District,NCESDist,year
1,9700000US0632070,"Redlands Unified School District, California",103753,75568,-999999999,-999999999,-999999999,-999999999,-999999999,-999999999,...,-999999999.0,-999999999.0,4.9,6.7,5.2,06,32070,Redlands Unified School District,0632070,2019
2,9700000US0632550,"West Contra Costa Unified School District, Cal...",205097,65715,32732,-999999999,44059,-999999999,51712,9171,...,3.5,7.1,3.3,3.2,2.7,06,32550,West Contra Costa Unified School District,0632550,2019
3,9700000US0633150,"Riverside Unified School District, California",215389,123698,14620,-999999999,19921,-999999999,44992,-999999999,...,6.8,-999999999.0,5.5,3.4,4.4,06,33150,Riverside Unified School District,0633150,2019
4,9700000US0633750,"Rowland Unified School District, California",87017,20198,-999999999,-999999999,41285,-999999999,21700,-999999999,...,7.3,-999999999.0,5.5,6.1,6.5,06,33750,Rowland Unified School District,0633750,2019
5,9700000US0633840,"Sacramento City Unified School District, Calif...",274468,146375,29800,-999999999,49604,-999999999,27093,16922,...,7.5,8.3,7.3,5.5,5.5,06,33840,Sacramento City Unified School District,0633840,2019


In [41]:
withoutage.to_csv("/Users/nataligracia/git/ca-school-enrollment-trend/Unified2019.csv")

In [42]:
#Rearrange columns in list
#['NCESDist','District'] + list(without_annotation_df.columns[2:])

In [43]:
#Rearrange columns in dataframe
#final = without_annotation_df[['NCESDist','District','Geography','NAME'] + list(without_annotation_df.columns[2:])]
#final